In [1]:
from queue import PriorityQueue

In [2]:
class Library:
    def __init__(self, id, n_books, signup_time, books_per_day, books):
        self.id = id
        self.n_books = n_books                  # number of books
        self.signup_time = signup_time          # signup time
        self.books_per_day = books_per_day      # books per day
        self.books = books
    
    def __str__(self):
        return f"{self.id} {self.n_books} {self.signup_time} {self.books_per_day} {self.books}"
    
    """
    def calculate_score_per_day(self):
        score = 0
        green_books = 0
        pq = PriorityQueue()
        while (not self.books.empty()):
            book = self.books.get()
            if (book.status == 0):
                score += book.score
                green_books += 1
                pq.put(book)
            else:
                break
        
        while (not pq.empty()):
            self.books.put(pq.get())
        
        if (green_books == 0):
            return (0, 0)
        
        time_taken = self.signup_time + green_books / self.books_per_day

        return (score * self.books_per_day / (green_books * time_taken), green_books) """
    
    def calculate_possible_score(self, days_left):
        library_available_days = days_left - self.signup_time
        if (library_available_days <= 0):
            return 0
    
        books_to_scan = library_available_days * self.books_per_day

        library_books_score = 0

        pq = PriorityQueue()

        counter = 0

        while not self.books.empty():
            if (counter == books_to_scan):
                break

            book = self.books.get()
            
            if (book.status == 0):
                pq.put(book)
                library_books_score += book.score
                counter += 1
            else:
                break

        while not pq.empty():
            self.books.put(pq.get())

        return (library_books_score / self.signup_time, counter)

        

    def __lt__(self, other):
        global days_left
        spd1 = self.calculate_possible_score(days_left)
        spd2 = other.calculate_possible_score(days_left)

        if (spd1[0] == spd2[0]):
            return spd1[1] > spd2[1]
        return spd1[0] > spd2[0]

class Book:
    def __init__(self, id, score):
        self.id = id
        self.score = score
        self.status = 0
    
    def __str__(self):
        return f"{self.id} {self.score}" 
    
    def __lt__(self, other):
        if (self.status == other.status):
            return self.score > other.score
        return self.status < other.status
    
    def turn_yellow(self):
        self.status = 1

    def turn_red(self):
        self.status = 2 
    
class Singleton:
    def __init__(self, total_books, total_libs, total_days, scores, libs):
        self.total_books = total_books  # total number of books
        self.total_libs = total_libs  # total number of libraries
        self.total_days = total_days  # total number of days
        self.scores = scores
        self.libs = libs

    def __str__(self):
        return f"{self.total_books} {self.total_libs} {self.total_days} {self.libs}"


In [3]:
libs_dict = {}
books_dict = {}

def read_file(filename):

    f = open(filename, "r")

    line = f.readline()

    values = [int(x) for x in line.split()]

    total_books = values[0]
    total_libs = values[1]
    total_days = values[2]

    line = f.readline()
    scores = [int(x) for x in line.split()]

    for i in range(len(scores)):
        books_dict[i] = Book(i, scores[i])

    libs = PriorityQueue()
    i = 0
    while True:
        line = f.readline()    
        values = [int(x) for x in line.split()]

        if len(values) == 0:
            break
        
        line = f.readline()
        books = [int(x) for x in line.split()]

        q = PriorityQueue()
        for b in books:
            q.put(books_dict[b])

        lib = Library(i, values[0], values[1], values[2], q)
        
        libs.put(lib)
        libs_dict[i] = lib
        i += 1

    f.close()

    return total_books, total_libs, total_days, scores, libs

def read_total_days(filename):

    f = open(filename, "r")
    line = f.readline()

    values = [int(x) for x in line.split()]
    return values[2]

In [4]:
def solve(att):
    solution = []
    days = 0
    scanned_books = set()
    libs_signed_up = set()
    while (not att.libs.empty()):
        lib = att.libs.get()
        days += lib.signup_time
        if (days >= att.total_days):
            break
        days_left = att.total_days - days
        books_left = days_left * lib.books_per_day
        books = []
        while (not lib.books.empty()):
            book = lib.books.get()
            if (book.status == 0):
                books.append(book.id)
                if (len(books) <= books_left):
                    scanned_books.add(book.id)
        
        solution.append((lib.id, books))
        libs_signed_up.add(lib.id)

    score = 0
    for book in scanned_books:
        score += books_dict[book].score

    return solution, score
            


In [5]:
data_files = [
    "data/a_example.txt",
    "data/b_read_on.txt",
    "data/c_incunabula.txt",
    "data/d_tough_choices.txt",
    "data/e_so_many_books.txt",
    "data/f_libraries_of_the_world.txt"
]

In [7]:
# final_score = 0

# # delete all contents of the "initial_solutions.txt" file
# f = open("initial_solutions.txt", "w")
# f.close()

# for file in data_files:
#     libs_dict = {}
#     books_dict = {}
#     days_left = read_total_days(file)
#     total_books, total_libs, total_days, scores, libs = read_file(file)
#     att = Singleton(total_books, total_libs, total_days, scores, libs)
#     print(f"File {file} ingested")

#     solution, score = solve(att)
#     final_score += score

#     write_solution(solution, "initial_solutions.txt")
    
#     print(f"Solution for file {file}: {solution}")
#     print(f"Score: {score}")

# print(f"Final score: {final_score}")

In [6]:
# function that returns all possible combinations of library swaps from an initial solution
def library_swap(initial_solution):
    l = len(initial_solution)

    new_solutions = []
    for i in range(1, l - 1):
        for k in range(i + 1, l):
            new_solutions.append(initial_solution[:i] + [initial_solution[k]] + initial_solution[i + 1:k] + [initial_solution[i]] + initial_solution[k + 1:])
    
    return new_solutions
            

In [9]:
new_solutions = library_swap([(0, [3, 4, 2, 1, 0]), (1, [3, 5, 2, 0])])

In [7]:
def calculate_score(solution, scores, total_days):
    score = 0
    scanned_books = set()
    for lib in solution:
        library = libs_dict[lib[0]]
        books = lib[1]
        total_days = total_days - library.signup_time
        
        if (total_days <= 0):
            break

        books_left = total_days * library.books_per_day
        for i in range(min(len(books), books_left)):
            if books[i] not in scanned_books:
                score += scores[books[i]]
                scanned_books.add(books[i])

    return score

In [11]:
# file = data_files[4]

# libs_dict = {}
# books_dict = {}
# days_left = read_total_days(file)
# total_books, total_libs, total_days, scores, libs = read_file(file)
# att = Singleton(total_books, total_libs, total_days, scores, libs)
# print(f"File {file} ingested")

# solution, score = solve(att)

# new_solutions = library_swap(solution)

# for s in new_solutions:
#     new_score = calculate_score(s, att.scores, att.total_days)
#     if (new_score > score):
#         print(new_score, s)

In [8]:
def write_solution(solution, filename):
    with open(filename, "a") as f:
        f.write(f"{len(solution)}\n")
        for lib in solution:
            f.write(f"{lib[0]} {len(lib[1])}\n")
            f.write(" ".join([str(x) for x in lib[1]]) + "\n")
        
        
    

In [27]:
import math
import random


def get_neighbors(initial_solution):
    l = len(initial_solution)

    new_solutions = []
    for i in range(1, l - 1):
        for k in range(i + 1, l):
            new_solutions.append(initial_solution[:i] + [initial_solution[k]] + initial_solution[i + 1:k] + [initial_solution[i]] + initial_solution[k + 1:])
    
    return new_solutions
            

# basic Hill Climbing
def hill_climbing_basic(solution, scores, total_days):
    best_score = calculate_score(solution, scores, total_days)
    l = len(solution)
    best_solution = solution
    there_is_best = True
    while there_is_best:
        there_is_best = False
        for i in range(1, l - 1):
            for k in range(i + 1, l):
                n = best_solution[:i] + [best_solution[k]] + best_solution[i + 1:k] + [best_solution[i]] + best_solution[k + 1:]
                s = calculate_score(n, scores, total_days)
                if s > best_score:
                    best_score = s
                    best_solution = n
                    there_is_best = True
                    break

    return best_solution, best_score

# Hill Climbing with Steepest Ascent    
def hill_climbing_steepest(solution, scores, total_days):
    best_score = calculate_score(solution, scores, total_days)
    l = len(solution)
    best_solution = solution
    there_is_best = True
    while there_is_best:
        there_is_best = False
        for i in range(1, l - 1):
            for k in range(i + 1, l):
                n = best_solution[:i] + [best_solution[k]] + best_solution[i + 1:k] + [best_solution[i]] + best_solution[k + 1:]
                s = calculate_score(n, scores, total_days)
                if s > best_score:
                    best_score = s
                    best_solution = n
                    there_is_best = True

    return best_solution, best_score

# Simulated Annealing
def simulated_annealing(solution, scores, total_days, max_iter, T, alpha):
    score = calculate_score(solution, scores, total_days)
    best_score = score
    best_solution = solution
    current_solution = solution
    current_score = score
    for _ in range(max_iter):
        i = random.randint(0, len(current_solution) - 2)
        k = random.randint(i + 1, len(current_solution) - 1)
        neighbor = current_solution[:i] + [current_solution[k]] + current_solution[i + 1:k] + [current_solution[i]] + current_solution[k + 1:]
        new_score = calculate_score(neighbor, scores, total_days)
        delta = new_score - current_score
        if delta > 0:
            current_solution = neighbor
            current_score = new_score
            if new_score > best_score:
                best_score = new_score
                best_solution = neighbor
        else:
            if random.random() < math.exp(delta / T):
                current_solution = neighbor
                current_score = new_score
        T = T * alpha

    return best_solution, best_score

# Tabu Search 
def tabu_search(solution, scores, total_days, max_iter, tenure):
    score = calculate_score(solution, scores, total_days)
    best_score = score
    best_solution = solution
    current_solution = solution
    current_score = score
    tabu_list = []
    for _ in range(max_iter):
        i = random.randint(0, len(current_solution) - 2)
        k = random.randint(i + 1, len(current_solution) - 1)
        neighbor = current_solution[:i] + [current_solution[k]] + current_solution[i + 1:k] + [current_solution[i]] + current_solution[k + 1:]
        new_score = calculate_score(neighbor, scores, total_days)
        if neighbor in tabu_list or new_score <= current_score*0.9:
            continue
        if new_score > best_score:
            best_score = new_score
            best_solution = neighbor
        current_solution = neighbor
        current_score = new_score
        tabu_list.append(neighbor)
        if len(tabu_list) > tenure:
            tabu_list.pop(0)

    return best_solution, best_score


# Simulated Annealing with Tabu Search
def simulated_annealing_tabu_search(solution, scores, total_days, max_iter, T, alpha, tenure):
    score = calculate_score(solution, scores, total_days)
    best_score = score
    best_solution = solution
    current_solution = solution
    current_score = score
    tabu_list = []
    for _ in range(max_iter):
        i = random.randint(0, len(current_solution) - 2)
        k = random.randint(i + 1, len(current_solution) - 1)
        neighbor = current_solution[:i] + [current_solution[k]] + current_solution[i + 1:k] + [current_solution[i]] + current_solution[k + 1:]
        new_score = calculate_score(neighbor, scores, total_days)
        if neighbor in tabu_list:
            continue
        delta = new_score - current_score
        if delta > 0:
            current_solution = neighbor
            current_score = new_score
            if new_score > best_score:
                best_score = new_score
                best_solution = neighbor
        else:
            if random.random() < math.exp(delta / T):
                current_solution = neighbor
                current_score = new_score
        
        tabu_list.append(neighbor)
        if len(tabu_list) > tenure:
            tabu_list.pop(0)
        T = T * alpha

    return best_solution, best_score


file = data_files[4]

libs_dict = {}
books_dict = {}
days_left = read_total_days(file)
total_books, total_libs, total_days, scores, libs = read_file(file)
att = Singleton(total_books, total_libs, total_days, scores, libs)
print(f"File {file} ingested")

solution, score = solve(att)
print("initial -> ", score, solution)

# new_solution, new_score = hill_climbing_basic(solution, scores, total_days)
# print("HC Basic -> ", new_score, new_solution)

# new_solution, new_score = hill_climbing_steepest(solution, scores, total_days)
# print("HC Steepest -> ", new_score, new_solution)

# new_solution, new_score = simulated_annealing(solution, scores, total_days, 5000, 150, 0.99)
# print("SA -> ", new_score, new_solution)

# new_solution, new_score = tabu_search(solution, scores, total_days, 5000, 10)
# print("TS -> ", new_score, new_solution)

new_solution, new_score = simulated_annealing_tabu_search(solution, scores, total_days, 5000, 150, 0.99, 10)
print("SA + TS -> ", new_score, new_solution)

File data/e_so_many_books.txt ingested
initial ->  4135783 [(602, [15550, 23967, 79903, 69364, 63011, 33121, 30097, 56764, 17387, 87189, 90748, 14932, 85484, 36501, 75429, 12166, 75939, 46139, 30426, 77427, 73950, 88461, 97434, 2233, 94172, 65637, 67152, 93832, 61389, 69677, 84684, 23417, 63771, 90624, 17610, 17583, 15858, 35969, 53812, 2260, 43991, 10745, 54700, 88856, 12549, 4872, 62860, 36168, 84258, 88739, 59356, 42378, 56331, 54410, 1482, 51871, 98352, 53923, 94920, 20347, 79835, 43645, 75515, 27816, 96765, 4179, 93225, 59266, 6387, 39709, 11215, 3788, 94426, 21673, 21240, 73206, 43331, 6074, 50611, 24072, 45824, 26052, 41607, 71998, 30318, 30370, 88512, 51658, 64194, 445, 21197, 83262, 7842, 87651, 77416, 96138, 59000, 71518, 23413, 77513, 90433, 27285, 78680, 62851, 37305, 81286, 34878, 37805, 51856, 84370, 48377, 77360, 95251, 13815, 75527, 79191, 87183, 43859, 98340, 88926, 60800, 20301, 34288, 24819, 68169, 65747, 28556, 43277, 37733, 70857, 18259, 86161, 50120, 11370, 71367,

In [9]:
data_files = [
    'data/a_example.txt',
    'data/b_read_on.txt',
    'data/c_incunabula.txt',
    'data/d_tough_choices.txt',
    'data/e_so_many_books.txt',
    'data/f_libraries_of_the_world.txt'
]

def write_score(score, filename):
    with open(filename, "a") as f:
        f.write(f"{score} ")


def write_solution(solution, filename):
    with open(filename, "a") as f:
        f.write(f"{len(solution)}\n")
        for lib in solution:
            f.write(f"{lib[0]} {len(lib[1])}\n")
            f.write(" ".join([str(x) for x in lib[1]]) + "\n")

## **Initial Script**

In [12]:
f = open("solutions/initial_solutions.txt", "w")
f.close()

f = open("solutions/initial_scores.txt", "w")
f.close()

for i in range(len(data_files)):
    days_left = read_total_days(data_files[i])
    total_books, total_libs, total_days, scores, libs = read_file(data_files[i])
    att = Singleton(total_books, total_libs, total_days, scores, libs)

    solution, score = solve(att)

    write_solution(solution, "solutions/initial_solutions.txt")
    write_score(score, "solutions/initial_scores.txt")